In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib as mpl
import seaborn as sns
import torch
import wandb
from matplotlib import pyplot as plt
import pandas as pd
import typing

import opf
import opf.utils
import opf.dataset
from opf.hetero import HeteroGCN
from opf.modules import OPFDual, OPFDual

mpl.rcParams["text.usetex"] = False

sns.set_style("white")
sns.set_context("paper")

torch.set_float32_matmul_precision("high")

In [ ]:
run_uri = "wandb://damowerko/opf/xtfljzbn"

In [ ]:
base, path = run_uri.split("://")
if base != "wandb":
    raise ValueError("Only wandb runs are supported")

api = wandb.Api()
run = api.run(path)
run.config.update(data_dir="../data")

# initialize data
config = run.config
# config.update(batch_size=128)
dm = opf.dataset.CaseDataModule(**config)
dm.setup("test")
assert dm.test_dataset is not None

# initialize model and initialize lazy layers
model = OPFDual(**run.config, model=HeteroGCN(dm.metadata(), -1, 4, **config))
model(dm.test_dataset[0])

# load checkpoint
with TemporaryDirectory() as tmpdir:
    artifact = api.artifact(f"damowerko/opf/model-{run.id}:best")
    checkpoint_path = artifact.download(root=tmpdir)
    checkpoint = torch.load(Path(checkpoint_path) / "model.ckpt", map_location="cpu")
    model.load_state_dict(checkpoint["state_dict"], strict=True)

In [ ]:
save_dir = Path(f"../figures/dual/{run.config['case_name']}/")
save_dir.mkdir(parents=True, exist_ok=True)


def save(title: str, fig=plt):
    # fig.savefig(save_dir / f"{title}.png")
    fig.savefig(save_dir / f"{title}.pdf")

In [ ]:
from opf import powerflow as pf
import h5py
from pathlib import Path
from tqdm.notebook import tqdm

metrics = []
with torch.no_grad():
    for batch in tqdm(dm.test_dataloader()):
        variables = model.forward(batch)
        variables = model.project_powermodels(
            variables, batch.powerflow_parameters, clamp=True
        )
        for i in range(batch.data.num_graphs):
            _variables = variables[i]
            cost = model.cost(_variables, batch.powerflow_parameters)
            constraints = model.constraints(_variables, batch.powerflow_parameters)
            metrics.append(
                {
                    k: v.item()
                    for k, v in model.metrics(cost, constraints, "test", True).items()
                }
            )
df = pd.DataFrame(metrics)

In [ ]:
with h5py.File(Path(batch.powerflow_parameters.casefile).with_suffix(".h5"), "r") as f:
    df["acopf/cost"] = f["objective"][-1000:] / batch.powerflow_parameters.reference_cost  # type: ignore

# Summary

In [ ]:
display(
    df[
        [
            "test/equality/error_max",
            "test/equality/bus_active_power/error_max",
            "test/equality/bus_reactive_power/error_max",
            "test/inequality/error_max",
            "test/inequality/active_power/error_max",
            "test/inequality/reactive_power/error_max",
            "test/inequality/voltage_magnitude/error_max",
            "test/inequality/forward_rate/error_max",
            "test/inequality/backward_rate/error_max",
            "test/inequality/voltage_angle_difference/error_max",
        ]
    ].max()
)

print(
    f"""
GNN Cost: {df["test/cost"].mean():0.4f}

Maximum violation rate: {df["test/inequality/rate"].max():0.4f}
Rate of any violation: {(df["test/inequality/error_max"] > 1e-4).sum() / len(df):0.4f}
IPOPT Cost: {df["acopf/cost"].mean():0.4f}
GNN/IPOPT: {(df["test/cost"]/df["acopf/cost"]).mean():0.4f}
"""
)

# Histograms

In [ ]:
from opf.utils import FlowLayout

aspect = 1.618
ylabel = "Count / # of samples"
kwargs = dict(
    bins=10,
    stat="proportion",
    aspect=aspect,
    height=3.5 / aspect,
)


sns.displot(df, x="test/inequality/error_max", **kwargs)
plt.xlabel("Max inequality error")
plt.ylabel(ylabel)
save("error_max")

sns.displot(df, x="test/inequality/error_mean", **kwargs)
plt.xlabel("Mean inequality error")
plt.ylabel(ylabel)
save("error_mean")

# Cost improvement
df["test/cost/improvement"] = df["test/cost"] / df["acopf/cost"]
df["violation"] = df["test/inequality/rate"] > 1e-3
sns.displot(df[~df["violation"]], x="test/cost/improvement", **kwargs)
plt.xlabel("GNN / IPOPT cost ratio")
plt.ylabel(ylabel)
save("costs")

# map variable names to series names
fmt = "test/inequality/%s/error_max"
hist_dict = {
    "equality": ["test/equality/bus_active_power/error_max"],
    "gen": [fmt % "active_power", fmt % "reactive_power"],
    "vm": [fmt % "voltage_magnitude"],
    "rate": [fmt % "forward_rate", fmt % "backward_rate"],
    "vad": [fmt % "voltage_angle_difference"],
}

sns.displot(df["test/equality/bus_active_power/error_max"], **kwargs)
plt.xlabel("Bus power equality error")
plt.ylabel(ylabel)
save("error_equality")

power_df = df.melt(value_vars=[fmt % "active_power", fmt % "reactive_power"])
sns.displot(power_df, x="value", **kwargs)
plt.xlabel("Generated power error")
plt.ylabel(ylabel)
save("error_gen")

sns.displot(df[fmt % "voltage_magnitude"], **kwargs)
plt.xlabel("Voltage magnitude error")
plt.ylabel(ylabel)
save("error_vm")

flow_df = df.melt(value_vars=[fmt % "forward_rate", fmt % "backward_rate"])
sns.displot(flow_df, x="value", **kwargs)
plt.xlabel("Power rate limit error")
plt.ylabel(ylabel)
save("error_rate")

sns.displot(df[fmt % "voltage_angle_difference"], **kwargs)
plt.xlabel("Voltage angle difference error")
plt.ylabel(ylabel)
save("error_vad")

FlowLayout().all_open()

# Visualizing Violations

In [ ]:
sort_term = "test/inequality/error_max"
quantile = 1.0

s = df[sort_term]
index = int((s.sort_values()[::-1] <= s.quantile(quantile)).idxmax())
print(sort_term, s[index])
print("Idx", index)
df.iloc[index][
    [
        "test/cost",
        "acopf/cost",
        "test/equality/bus_active_power/error_max",
        "test/equality/bus_reactive_power/error_max",
        "test/equality/bus_reference/error_max",
        "test/inequality/error_max",
        "test/inequality/active_power/error_max",
        "test/inequality/reactive_power/error_max",
        "test/inequality/voltage_magnitude/error_max",
        "test/inequality/forward_rate/error_max",
        "test/inequality/backward_rate/error_max",
        "test/inequality/voltage_angle_difference/error_max",
    ]
]

In [ ]:
# perform inference on the test set
from opf.plot import plot_constraints
import opf.powerflow as pf

with torch.no_grad():
    dm.setup("test")
    dataset = dm.test_dataset
    # assert isinstance(dataset, opf.dataset.StaticGraphDataset)
    data = dataset[index]
    variables = model.float()(data)
    # substitute_equality was True
    variables, _, _, _ = model._step_helper(
        variables, data.powerflow_parameters, project_powermodels=True
    )
    constraints = pf.build_constraints(variables, data.powerflow_parameters)
    plots = plot_constraints(constraints)
for name in plots:
    name = typing.cast(str, name)
    save(f"constraint_{name.replace('/', '_')}", fig=plots[name])